## 如何使用R的dplyr函數包去處理股價資料
### 程式撰寫：Yen-Ting, Su
### 2017.11.24

### 參考學習資源：

[dplyr函數包官方說明檔](https://cran.r-project.org/web/packages/dplyr/dplyr.pdf)

[Introduction to dplyr](https://cran.r-project.org/web/packages/dplyr/vignettes/dplyr.html)

In [ ]:
library(dplyr)
library(quantmod)

讀取20170101至20171031的台灣上市日頻股價資訊

資料以 R 軟體既有的 data.frame 格式儲存

In [ ]:
load("stockPriceData.Rdata")

資料欄位內容：
1. code: 股票代碼
2. name: 股票名稱
3. date: 日期
4. open: 開盤價
5. high: 最高價
6. low: 最低價
7. close: 收盤價
8. tradeVolume: 成交量

In [ ]:
head(stockPriceData, 10)

In [ ]:
class(stockPriceData)

首先將資料格式轉為 dplyr 的資料表，使用 class() 函數可以看到， dplyr 的資料表同時具有3種格式，其中包含 R 原生的 data.frame 格式，所以 R 原本內建的資料表處理函數也可以一起使用。

In [ ]:
stockPriceData <- stockPriceData %>% as_data_frame() 

In [ ]:
class(stockPriceData)

# filter()

利用 filter() 可以從資料表中篩選出想要的樣本，類似使用 R 的 which()。

* 篩選範例：選出1101台泥股票的資料

In [ ]:
stockPriceData %>% filter(code==1101)

* 多條件交集篩選，以 & 連接條件：選出1101台泥股票在20170103的資料

In [ ]:
stockPriceData %>% filter(code==1101 & date==20170103)

* 另一種多條件交集篩選寫法， 以 , 連接條件：選出1101台泥股票在20170103的資料

In [ ]:
stockPriceData %>% filter(code==1101, date==20170103)

* 聯集條件篩選，以 | 連接條件：選出1101台泥股票和1102亞泥股價資料

In [ ]:
stockPriceData %>% filter(code==1101 | code==1102)

* 另一種多條件聯集篩選寫法：選出1101台泥股票、1102亞泥和1103嘉泥股價資料

In [ ]:
stockPriceData %>% filter(code==c(1101,1102,1103))

# arrange()

利用 arrange() 可以對資料表依指定欄位做排序，即為 R 的 order() 函數功能。

* 排序寫法：依照股票代碼由小到大排序

In [ ]:
stockPriceData %>% arrange(code)

* 多條件排序寫法，按輸入的欄位名稱順序進行排序：以股票代碼優先由小到大排序，日期為次要由小到大排序

In [ ]:
stockPriceData %>% arrange(code, date)

* 由大到小排序寫法：依照日期由大到小排序

In [ ]:
stockPriceData %>% arrange(desc(date))

* 多個欄位排序寫法：依日期由大到小優先排序，再依日期由小到大做次要排序

In [ ]:
stockPriceData %>% arrange(desc(code), date)

# mutate()

利用 mutate() 可以在資料表中新增資料欄位。

* 新增一欄今日股票的開盤價和收盤價的差距金額

In [ ]:
stockPriceData %>% mutate(diffOpenClose=close-open)

* 同時新增兩個欄位，利用 , 來區別：分別為今日股票的 開盤價和收盤價 及 最高價和最低價 差距金額

In [ ]:
stockPriceData %>% mutate(diffOpenClose=close-open, diffHighLow=high-low)

# transmute()

tansmute() 和 mutate() 類似，都是計算出新的欄位資料，差異點為 transmute() 只會呈現指定到的欄位。

* 新增開盤價與收盤價、最高價與最低價差距金額欄位，並只選出股票代碼、股票名稱和日期欄位

In [ ]:
stockPriceData %>% transmute(code, name, date, diffOpenClose=close-open, diffHighLow=high-low)

# select()

select() 為選取資料表要呈現的欄位，與 SQL 用的 select 概念是一樣的，可用來調整欄位的順序。

* 選取日期、股票名稱、收盤價

In [ ]:
stockPriceData %>% transmute(date, name, close)

# rename()

rename() 可以用來修改資料表的欄位名稱。

* 將資料表的 code 欄位名稱改為 stockCode、 date 欄位名稱改為 tradeDate

In [ ]:
stockPriceData %>% rename(stockCode=code, tradeDate=date)

# distinct()

distinct() 即是 R 語言內的 unique() 函數，過濾掉重複的資料。

* 建立資料表內不重複的交易日期清單

In [ ]:
stockPriceData %>% distinct(date)

* distinct() 可以選取多個欄位:建立資料表內不重複的股票代碼及名稱清單

In [ ]:
stockPriceData %>% distinct(code, name)

# group_by()

group_by() 是依指定欄位的資料值，分出各個子資料集，接下來的 dplyr 函數操作都是以各個子資料集為單位進行計算與整理。

在股價資料中，利用 group_by() 指定股票代碼，即可一次計算所有股票的報酬率、技術指標值或是做條件判斷等，待會後面的範例會做示範。

在執行 group_by() 後，在 R 中直接讀取資料變數，會在第二行呈現目前的分組依據欄位。

以下表為例，此資料表是已股票代碼為分組依據，總共分成928組


In [ ]:
groupData <- stockPriceData %>% group_by(code)
print(groupData)

若想要取消群組，只要再輸入一次group_by()，括號內不指定欄位即可

In [ ]:
noGroupData <- groupData %>% group_by()
print(noGroupData)

# summarise

通常與 group_by() 及其他 R 函數配合，可對各子資料集做敘述統計學分析。

* 計算各支股票在過去期間的平均成交量、最大的收盤價格及最小的收盤價格

In [ ]:
stockPriceData %>% group_by(code) %>% summarise(meanVolume=mean(tradeVolume), maxClose=max(close), minClose=min(close))

# slice()

slice() 主要用來選取資料表所需的列數

* 選取資料表前10筆樣本

In [ ]:
stockPriceData %>% slice(1:10)

* 選取每支股票的前10筆樣本: 透過group_by搭配slice

In [ ]:
stockPriceData %>% group_by(code) %>% slice(1:5)

# row_number()

row_number() 可對樣本進行編號

* 對資料表所有樣本依序進行編號

In [ ]:
stockPriceData %>% mutate(num=row_number())

# n()

計算樣本總數

* 計算資料表樣本總數

In [ ]:
stockPriceData %>% mutate(totalNums=n())

* 計算各支股票交易日數

In [ ]:
stockPriceData %>% group_by(code) %>% summarise(tradeDayNums=n())

# left_join()

在數據處理中，常會發生所需的數據會來自不同的表，需要加以整併，這時候就是併表函數使用的時機。

當有兩張表時，一張為左表，另一張為右表，若以左表做為被併的表(主表)，右表為併入的表，則稱為left join。

此函數與 R 的 merge() 函數類似。

此處只介紹最常用的left join，其他併表函數可參考：[Hiroaki Yutani@Twitter 製作的 cheat sheet](https://twitter.com/yutannihilation/status/551572539697143808)

* 以下示範如何整理出1101台泥和1102亞泥收盤價做為欄位的表

In [ ]:
stock_1101 <- stockPriceData %>% filter(code==1101) %>% select(date, close_1101=close) # 製作1101台泥的收盤價資料表
stock_1102 <- stockPriceData %>% filter(code==1102) %>% select(date, close_1102=close) # 製作1102亞泥的收盤價資料表
joinData <- stock_1101 %>% left_join(stock_1102, by=c("date"="date"))                  # 併表(此處以1101為主表)
joinData

# bind_rows() & bind_cols()

bind_rows() 和 bind_cols() 等同於 R 原本的 rbind() 和 cbind() 函數。

* 以1101台泥股價表和1102亞泥股價表進行示範

In [ ]:
# 製作1101台泥的收盤價資料表
stock_1101 <- stockPriceData %>% filter(code==1101 & date<=20170109) %>% select(date, close) 

# 製作1102亞泥的收盤價資料表
stock_1102 <- stockPriceData %>% filter(code==1102 & date<=20170109) %>% select(date, close)

列併表

In [ ]:
bind_rows(stock_1101, stock_1102)

欄併表

In [ ]:
bind_cols(stock_1101, stock_1102)

## 應用範例1: 計算移動平均線範例

此範例示範如何快速計算每支股票的移動平均線資料

In [ ]:
stockPriceData %>%        
arrange(code, date) %>%      # 按股票代號及日期進行排序
group_by(code) %>%           # 以股票代號為群組
filter(n()>60) %>%           # 過濾交易日數不足股票:避免交易日數低於60日無法計算移動平均線Bug
mutate(MA5=SMA(close,5),     # 計算5日移動平均線
       MA20=SMA(close,20),   # 計算20日移動平均線
       MA60=SMA(close,60))   # 計算60日移動平均線

## 應用範例2: 計算各支股票報酬率

此範例示範如何快速計算每支股票的日報酬率

In [ ]:
stockPriceData %>%        
arrange(code, date) %>%        # 按股票代號及日期進行排序
group_by(code) %>%             # 以股票代號為群組
mutate(lagClose=lag(close,1),  # 昨日收盤價:lag股價一期
       ret=close/lagClose-1)   # 報酬率=今日收盤價/昨日收盤價-1

## 應用範例3: 尋找黃金交叉進場點

此範例示範如何找到各支股票的5日均線向上突破10日均線黃金交叉位置

In [ ]:
stockPriceData %>%        
arrange(code, date) %>%          # 按股票代號及日期進行排序
group_by(code) %>%               # 以股票代號為群組
filter(n()>10) %>%               # 過濾交易日數不足股票:避免交易日數低於10日無法計算移動平均線Bug
mutate(MA5=SMA(close,5),         # 計算5日移動平均線
       MA10=SMA(close,10),       # 計算10日移動平均線
       lagMA5=lag(MA5,1),        # 昨日5日移動平均線
       lagMA10=lag(MA10,1)) %>%  # 昨日10日移動平均線
filter(lagMA5<lagMA10, MA5>MA10) # 黃金交叉條件(昨日5日均線<昨日10日均線，今日5日均線>今日10日均線)